\title{Problem Set 3}

\author{Pawel Langer - worked with Laszlo}

\date{November 2, 2017}


\maketitle

\section*{State space representation}

In order to solve the problem, we first stack all the equations into canonical the canonical form:
\begin{equation*}
A(\theta)X_{t}=B(\theta)E_{t}X_{t+1}+C(\theta)X_{t-1}+D(\theta)\eta_{t}
\end{equation*}
With the matrices as follows:
\begin{equation*}
A=
\begin{bmatrix}
   1 & 0 & \sigma & -1 & 0 \\
  -\kappa & 1 & 0 & 0 & -1 \\
  -\phi_{x}(1-\rho_{i}) & -\phi_{i}(1-\rho_{i}) & 1 & 0 & 0 \\
  0 & 0 & 0 & 1 & 0 \\
  0 & 0 & 0 & 0 & 1 \\
\end{bmatrix}
\end{equation*}
\begin{equation*}
B=
\begin{bmatrix}
   1 & \sigma & 0 & 0 & 0 \\
   1 & \beta & 0 & 0 & 0 \\
   0 & 0 & 0 & 0 & 0 \\
   0 & 0 & 0 & 0 & 0 \\
   0 & 0 & 0 & 0 & 0 \\
\end{bmatrix}
,C=
\begin{bmatrix}
   0 & 0 & 0 & 0 & 0 \\
   0 & 0 & 0 & 0 & 0 \\
   0 & 0 & \rho_{i} & 0 & 0 \\
   0 & 0 & 0 & \rho_{g} & 0 \\
   0 & 0 & 0 & 0 & \rho_{u} \\
\end{bmatrix}
,D=
\begin{bmatrix}
   0 & 0 & 0 \\
   0 & 0 & 0 \\
   \sigma_{i} & 0 & 0 \\
   0 & \sigma_{g} & 0 \\
    0 & 0 & \sigma_{u} \\
\end{bmatrix}
\end{equation*}
Next we conjecture that the solution is linear:
\begin{equation*}
X_{t}=FX_{t-1}+G\eta_{t}
\end{equation*}\\
Since we can solve for $G$ once we know $F$. The $F$ matrix solves the following quadratic equation:
\begin{equation*}
BF^{2}-AF+C=0
\end{equation*}\\
We solve for $F$ using the Generalized Schur Decomposition. The observed variables $Y_{t}$ evolve according to: 
\begin{equation*}
Y_{t}=H X_{t}+J \zeta_{t}
\end{equation*}\\
with 
\begin{equation*}
H=
\begin{bmatrix}
   1 & 0 & 0 & 0 & 0 \\
  0 & 1 & 0 & 0 & 0 \\
  0 & 0 & 1 & 0 & 0 \\
 \end{bmatrix}
\end{equation*}
\begin{equation*}
J= \sigma_m
\begin{bmatrix}
   1 & 0 & 0 \\
  0 & 1 & 0  \\
  0 & 0 & 1 \\
 \end{bmatrix}
\end{equation*}

In [1]:
import numpy as np
from numba import jit
import scipy.linalg as linalg
import scipy.stats as stats
import scipy.optimize as optimize
import matplotlib.pyplot as plt
%matplotlib inline
#import warnings; warnings.simplefilter('ignore')
from tqdm import tqdm
import quandl as Quandl
import time
from scipy.stats import multivariate_normal

True parameter vector is:

In [3]:
param_true = np.array([2.09,0.98,2.25,0.65,0.81,0.98,0.93,0.34,3.16,0.51,0.19,0.65,0.24])

Corresponding to the parameters:
\begin{equation}
\tau, \kappa,\beta, \phi_{\pi} , \phi_{x}, \rho_i, \rho_g, \rho_z,r^{A},\pi^{Q},\gamma^{Q}, \sigma_i,\sigma_g,\sigma_z
\end{equation}
and the standard deviation $\sigma_m$ for the observations is set initially to:

In [4]:
param_guess = np.array([3.26,0.89,1.88,0.53,0.76,0.98,0.89,0.19,3.29,0.73,0.20,0.58,0.29])

Solving for the matrices and loading the data:

In [5]:
#Kalman filter for Tempered Particle Filter
no_st = 6
no_observables = 3
@jit
def eval_theta(param):
    tau, kappa, psi_pi,psi_y,rho_i,rho_g,rho_z,const_r, const_i,const_y, sigma_i,sigma_g,sigma_z = param
    beta = 1/(1+const_r/400) 
    #beta = 1
    no_st = 6 #Number of x states - underlying states
    no_observables = 3
    #The X_t matrix
    A = np.reshape([1,0,1/tau,-1,0,0,
                    -kappa,1,0,kappa,0,0,
                    -psi_y*(1 - rho_i), - psi_pi * (1-rho_i),1,(1-rho_i)*psi_y,0,0,
                   0,0,0,1,0,0,
                    0,0,0,0,1,0,
                   0,0,0,0,0,1],(no_st,no_st))
    #The X_t+1 matrix
    B = np.reshape([1, 1/tau, 0, -1, 1/tau,0,
                  0, beta, 0, 0, 0,0,
                  0, 0, 0, 0, 0,0,
                  0, 0, 0, 0, 0,0,
                  0, 0, 0, 0, 0,0,
                   0, 0, 0, 0, 0,0],(no_st,no_st))
    #The X_t-1 matrix
    C = np.reshape([0, 0, 0, 0, 0,0,
                    0, 0, 0, 0, 0,0,
                    0, 0, rho_i, 0, 0,0,
                    0, 0, 0, rho_g, 0,0,
                    0, 0, 0, 0, rho_z,0,
                   1, 0, 0, 0, 0,0],(no_st,no_st))
    #The error matrix eta
    D = np.reshape([0, 0, 0,
                    0, 0, 0,
                   sigma_i, 0, 0,
                    0, sigma_g, 0,
                    0, 0, sigma_z,
                   0, 0, 0,],(no_st,no_observables))
    H = np.zeros((no_observables,no_st))
    H[0,0] = 1
    H[0,4] = 1
    H[0,5] = -1
    H[1,1] = 4
    H[2,2] = 4
    #J = np.eye(no_observables,no_observables)
    Const_obs = np.zeros((no_observables,1))
    Const_obs[0,0] = const_y
    Const_obs[1,0] = const_i
    Const_obs[2,0] = 4*const_y + const_i + const_r
    K = np.bmat([[np.zeros((no_st,no_st)), np.eye(no_st)], [-C, A]])
    L = np.bmat([[ np.eye(no_st),np.zeros((no_st,no_st))], [np.zeros((no_st,no_st)), B]])
    T,S,alppha,betta,Q,Z = linalg.ordqz(K,L,sort='iuc')
    Z = Z.T
    #print(1)
    #F = Z[(no_st):(2*no_st),0:no_st] @ linalg.inv(Z[0:no_st,0:no_st])
    if (linalg.det(Z[0:no_st,0:no_st]) != 0):
        F = (linalg.solve(Z[0:no_st,0:no_st],Z[0:no_st,(no_st):(2*no_st)])).T
        #print(2)
        #G = np.linalg.inv(A - (B @ F)) @ D
        if(linalg.det(A - (B @ F)) != 0):
            G = linalg.solve(A - (B @ F),D)
        else:
            F = np.eye(no_st)
            G = F
    else:
        F = np.eye(no_st)
        G = F   
    J =np.eye(no_observables,no_observables)
    J[0,0] = 0.1160
    J[1,1] = 0.2942
    J[2,2] = 0.4476
    #return A
    err = B @ F @ F - A @ F + C 
    return F , G , H, J,Const_obs,err

In [6]:
@jit
def KF_filter(F,G,H,J,x_obs,Const_obs):
    T = x_obs.shape[0]
    no_state = F.shape[0]
    no_measurement = J.shape[0]
    X = np.zeros((no_state,1))  # Number of state variables
    #P = np.mean(J.diagonal()) * np.zeros((no_state,no_state))  #Initial variance guess
    P = linalg.solve_discrete_lyapunov(F,G @ G.T)
    lnp = 0
    G_quad = G @ G.T
    J_quad = (J @ J.T)
    I_d = np.eye(no_measurement,no_measurement)
    for t in range(x_obs.shape[0]):
        X = F @ X
        P = F @ P @ F.T + G_quad
        y = x_obs[t,None].T - H @ X - Const_obs
        H_P_multi = H @ P
        V = H_P_multi @ H.T + J_quad
        inv_V = linalg.cho_solve(linalg.cho_factor(V,lower=True),I_d )
        #print(linalg.det(P))
        lnp = lnp - 0.5 * (np.log(np.linalg.det(V)) + y.T @ inv_V @ y)
        X = X + P @ H.T @ inv_V @ y
        P = P - H_P_multi.T @ inv_V @ H_P_multi
    return lnp - 0.5 * T * no_measurement *  np.log(2*np.pi)  #With 2pi adjustment constant

In [6]:
def freq_likelihood(param,x_obs):
    F,G,H,J = eval_theta(param)
    eig_val,eig_vec = linalg.eig(F)
    if (np.all(np.absolute(eig_val) >=1)) ==0:
        res = KF_filter(F,G,H,J,x_obs)
    else:
        res = -np.inf
    return res

In [7]:
GDPt = Quandl.get("FRED/GDPC1",authtoken="5QphWABG_zpJsB5dy4yW",start_date="1983-04-01", end_date="2003-01-01")
GDPt1 = Quandl.get("FRED/GDPC1",authtoken="5QphWABG_zpJsB5dy4yW",start_date="1983-01-01", end_date="2002-10-01")
POPt = Quandl.get("FRED/CNP16OV", collapse="quarterly",authtoken="5QphWABG_zpJsB5dy4yW",start_date="1983-01-01", end_date="2002-10-01")
POPt1 = Quandl.get("FRED/CNP16OV", collapse="quarterly",authtoken="5QphWABG_zpJsB5dy4yW",start_date="1982-10-31", end_date="2002-07-31")
FFR = Quandl.get("FRED/FEDFUNDS", collapse="quarterly",authtoken="5QphWABG_zpJsB5dy4yW",start_date="1983-01-01", end_date="2002-10-01")
CPIt = Quandl.get("FRED/CPIAUCSL", collapse="quarterly",authtoken="5QphWABG_zpJsB5dy4yW",start_date="1983-01-01", end_date="2002-10-01")
CPIt1 = Quandl.get("FRED/CPIAUCSL", collapse="quarterly",authtoken="5QphWABG_zpJsB5dy4yW",start_date="1982-10-31", end_date="2002-07-31")
T = 80
x_obs = np.zeros((T,no_observables))
x_obs[:,0] = 100*(np.log(GDPt.values/POPt.values)-np.log(GDPt1.values/POPt1.values)).flatten()
x_obs[:,1] = 400*(np.log(CPIt.values/CPIt1.values)).flatten()
x_obs[:,2] = FFR.values.flatten()

NameError: name 'GDPt' is not defined

In [7]:
x_obs = np.loadtxt("DataRecession.txt", delimiter=",")
#x_obs = np.loadtxt("DataFREDcorrect.csv", delimiter=",")

\section*{Simulation}
Now we similate for 200 periods:

In [8]:
F,G,H,J,Const_obs,err = eval_theta(param_true)

In [9]:
Trueval = KF_filter(F,G,H,J, x_obs,Const_obs)
Trueval

array([[-246.67813913]])

In [10]:
def random_sample(mean,var,no_sample):
    return np.random.multivariate_normal(mean,var,no_sample)

In [204]:
#Notes at: https://www.dropbox.com/home/Econometrics%20Notes?preview=nyu_Bayes5_2017.pdf
def particle_filter(F,G,H,J,x_obs,Const_obs,no_simu = 10000):  #F,G,H,J matrix from Kalman filter computation
    T = x_obs.shape[0]
    no_state = F.shape[0]
    no_measurement = J.shape[0]
    X_mean = np.zeros((no_st,1)) #Initialize the particle cloud at 0
    V_x = linalg.solve_discrete_lyapunov(F,G @ G.T) #Initial Variance function solved from FVF'−F+G @ G.T=0
    P = V_x #+ 10 * np.eye(no_state,no_state) # P matrix is the initial variance
    lnp = 0 # Initialize likelihood value
    #Eta = np.reshape(random_sample(np.zeros(no_state), G @ G.T,no_simu*T),(T,no_simu,no_state),order='F') 
    Eta = np.reshape(np.random.multivariate_normal(np.zeros(no_state), G @ G.T,no_simu*T),(T,no_simu,no_state),order='F') 
    #Eta shock reshaped for the number similutaion and time periods, ordered by Fortran optimal code
    X = np.random.multivariate_normal(X_mean.flatten(),P,no_simu) #Initial X draw
    p_yx = np.zeros((X.shape[0]))
    Var_y = J @ J.T
    inv_Var_y = linalg.inv(Var_y)
    const_obs = np.tile(Const_obs.T,(no_simu,1))
    for t in range(T): #Loop over time periods and add up the likelihood
        X_next = X @ F.T + Eta[t,:,:]             #Propagate the paricle forward
        y_dev = x_obs[t,:] -X_next @ H.T - const_obs
        p_yx = np.exp( -0.5 * (np.log(np.linalg.det(Var_y)) + np.diag(( y_dev) @ inv_Var_y @ (y_dev).T))) 
        #Computing the likelihoood
        p_yy = p_yx.mean()
        lnp = np.log(p_yy) + lnp
        weights_val = p_yx/p_yy/no_simu #Giving weights to the observations
        multisample = np.random.multinomial(no_simu, weights_val.flatten(), size=1).flatten() #resampling based on weights
        X_next = np.repeat(X_next,multisample,axis = 0) #Creating a new sample based on the weights
        X = X_next.copy()
    return lnp - 0.5 * T * no_measurement *  np.log(2*np.pi)

In [210]:
particle_filter(F,G,H,J,x_obs,Const_obs,no_simu = 1000)

-710.11717537370441

In [ ]:
no_runs = 20
likelihood_val = np.zeros((no_runs,2))
for i in tqdm(range(no_runs)):
    t0 = time.time()
    likelihood_val[i,0] = particle_filter(F,G,H,J,x_obs,Const_obs,no_simu = 40000)
    t1 = time.time()
    likelihood_val[i,1] = t1-t0


In [31]:
Bias = np.mean(likelihood_val[:,0])
StDev = np.std(likelihood_val[:,0])
AvgTime = np.mean(likelihood_val[:,1])
print(Bias, StDev,AvgTime)

-465.64124294 34.9371181946 371.083233714


In [82]:
Trueval

array([[-246.67813913]])

In [273]:
def tempered_particle_filter(F,G,H,J,x_obs,Const_obs,no_simu,r_star = 2,phi_init = 0.2,markov_time = 1):  #F,G,H,J matrix from Kalman filter computation
    #mvn = multivariate_normal(0,) #create a multivariate Gaussian object with specified mean and covariance matrix
    #p = mvn.pdf(x) #evaluate the probability density at x
    T = x_obs.shape[0]
    no_state = F.shape[0]
    no_measurement = J.shape[0]
    X_mean = np.zeros((no_st,1)) #Initialize the particle cloud at 0
    V_x = linalg.solve_discrete_lyapunov(F,G @ G.T) #Initial Variance function solved from FVF'−F+G @ G.T=0
    P = V_x# + 10 * np.eye(5,5) # P matrix is the initial variance
    lnp = 0 # Initialize likelihood value
    Eta = np.reshape(np.random.multivariate_normal(np.zeros(no_measurement), np.eye(no_measurement),no_simu*T),(T,no_simu,no_measurement),order='F') 
    #Eta shock reshaped for the number similutaion and time periods, ordered by Fortran optimal code
    #Eta = np.reshape(np.random.multivariate_normal(np.zeros(no_state), G @ G.T,no_simu*T),(T,no_simu,no_state),order='F') 
    X =  np.random.multivariate_normal(X_mean.flatten(),np.eye(no_state),no_simu) @ linalg.cholesky(P).T
    #X = random_sample(X_mean.flatten(),P,no_simu) #Initial X draw
    p_yx = np.zeros((X.shape[0]))
    const_obs = Const_obs.T
    Var_y = J @ J.T
    Var_eps = G @ G.T 
    inv_Var_y = linalg.inv(Var_y)
    maxiter = 30 # max number of iterations in the while loop
    accept_shock = np.random.uniform(size = (markov_time,no_simu,T,maxiter))
    #mutate_shock = np.reshape(np.random.multivariate_normal(np.zeros(no_state),np.eye(no_state),no_simu*T*maxiter*markov_time),(T,no_simu,no_state,maxiter,markov_time),order='F')
    mutate_shock = np.reshape(np.random.multivariate_normal(np.zeros(no_measurement),np.eye(no_measurement),no_simu*T*maxiter*markov_time
                                           ),(T,no_simu,no_measurement,maxiter,markov_time),order='F')
    def eval_weights(phi,phi_prev,e_dev):
        return np.exp(-e_dev*(phi-phi_prev))*(phi/phi_prev)**(no_measurement/2)
    def eval_inef_ratio(phi,phi_prev,e_dev,r_star):
        eval_weights_tmp = eval_weights(phi,phi_prev,e_dev)
        return np.sum((eval_weights_tmp/eval_weights_tmp.mean())**2)/no_simu -r_star
        #print(np.mean(np.exp(-2*(phi-phi_prev) * e_dev))/(np.mean(np.exp(-(phi-phi_prev) * e_dev)))**2 - r_star)
        #return np.mean(np.exp(-2*(phi-phi_prev) * e_dev))/(np.mean(np.exp(-(phi-phi_prev) * e_dev)))**2 - r_star
    def eval_inef_ratio_init(phi,r_star,no_simu,e_dev):
        A1=(2 * np.pi)**(-no_measurement/2) *  np.exp(- phi* e_dev) * (np.linalg.det(inv_Var_y*phi))**(1/2)
        B1=A1.mean()
        return np.sum((A1/B1)**2)/no_simu - r_star
    def eval_weights_avg(eval_weights_tmp):
        return eval_weights_tmp/ np.mean(eval_weights_tmp)/no_simu
    def c_var_updater_mh(x):
        return 0.95 + 0.1 * np.exp(20 * (x-0.4))/(1 + np.exp(20 * (x-0.4)))
    
    for t in range(T): #Loop over time periods and add up the likelihood
        print(t)
        #const_obs = 0
        c_var = 0.3
        #phi_tmp1 = phi_init
        #phi_tmp2 = phi_init
        X_ft = X @ F.T
        X_next = X_ft + Eta[t,:,:] @ G.T           #Propagate the paricle forward
        #plt.hist(Eta[t,:,:])
        y_dev = x_obs[t,:] -X_next @ H.T - const_obs
        e_dev = 0.5 * np.diag(( y_dev) @ inv_Var_y @ (y_dev).T)
        phi_init1 = optimize.bisect(eval_inef_ratio_init,phi_init,1,args = (r_star,no_simu,e_dev))
        phi = phi_init1
        #print(phi)
        p_yx = np.exp(- phi* e_dev) * (np.linalg.det(inv_Var_y*phi))**(1/2)
        ##p_yx_true = np.exp( -0.5 * np.log(np.linalg.det(Var_y)) - e_dev) 
        #Computing the likelihoood
        p_yy = p_yx.mean()
        weights_val = p_yx/p_yy/no_simu #Giving weights to the observations
        multisample = np.random.multinomial(no_simu, weights_val.flatten(), size=1).flatten() #resampling based on weights
        X_next = np.repeat(X_next,multisample,axis = 0) 
        X = np.repeat(X,multisample,axis = 0)#Creating a new sample based on the weights
        Eta_mutate = np.repeat(Eta[t,:,:] ,multisample,axis = 0)#Creating a new sample based on the weights
        #X = X_next_sample.copy()
        iterate = 0
        lnp = np.log(p_yx.mean()) + lnp - 0.5 * no_measurement *  np.log(2*np.pi)
        #print(np.log(p_yx.mean()))
        while (phi <1):
        #while (iterate < 0):
            #print(iterate,phi,lnp)
            #print(phi,phi_tmp1,phi_tmp2)
            y_dev = x_obs[t,:] -X_next @ H.T - const_obs
            e_dev = 0.5 * np.diag(( y_dev) @ inv_Var_y @ (y_dev).T)
            if(eval_inef_ratio(1,phi,e_dev,r_star) <= 0):
                weights_lik = eval_weights(1,phi,e_dev)
                #phi_prev = phi
                phi = 1
            else:
                #phi_tmp = optimize.fsolve(eval_inef_ratio,0.5,args = (phi,e_dev,r_star))
                #phi_tmp = optimize.root(eval_inef_ratio,(phi +1)/6,args = (phi,e_dev,r_star))
                #phi_tmp = optimize.brentq(eval_inef_ratio,phi,1,args = (phi,e_dev,r_star))
                phi_tmp = optimize.bisect(eval_inef_ratio,phi,1,args = (phi,e_dev,r_star))
                #print(lnp,phi,phi_tmp)
                #weights_lik = eval_weights(phi_tmp.x,phi,e_dev)
                weights_lik = eval_weights(phi_tmp,phi,e_dev)
                #print(weights_lik)
                phi = phi_tmp 
                #phi = phi_tmp.x
            weights_val = eval_weights_avg(weights_lik)
            #print(weights_val.max())
            multisample = np.random.multinomial(no_simu, weights_val.flatten(), size=1).flatten() #resampling based on weights
            X_next = np.repeat(X_next,multisample,axis = 0) 
            X = np.repeat(X,multisample,axis = 0)#Creating a new sample based on the weights
            Eta_mutate = np.repeat(Eta_mutate ,multisample,axis = 0)#Creating a new sample based on the weights
            
            for mt in range(markov_time):
                #print('runnnn')
                X_ft = X @ F.T
                Eta_mutate_tmp = Eta_mutate + c_var * mutate_shock[t,:,:,iterate,mt]
                X_next_mt_sample = X_ft + Eta_mutate_tmp @ G.T          #Propagate the paricle forward
                y_dev_mt_sample = x_obs[t,:] -X_next_mt_sample @ H.T - const_obs
                e_dev_mt_sample = 0.5 * np.diag(( y_dev_mt_sample) @ ( inv_Var_y) @ (y_dev_mt_sample).T)
                p_yx_tmp =  - phi* e_dev_mt_sample
                #X_next_mt_past = X_ft + Eta_mutate             #Propagate the paricle forward
                y_dev_mt_past = x_obs[t,:] -X_next @ H.T - const_obs
                e_dev_mt_past = 0.5 * np.diag(( y_dev_mt_past) @ (  inv_Var_y) @ (y_dev_mt_past).T)
                p_yx_past =  -  phi* e_dev_mt_past
                pr_ratio = -0.5 * np.sum(Eta_mutate_tmp**2 -Eta_mutate**2,1)
                acceptance_rate = np.exp( p_yx_tmp - p_yx_past+pr_ratio)
                decision = acceptance_rate > accept_shock[mt,:,t,iterate]
                tiled_decision = np.tile(decision,(no_measurement,1)).T
                tiled_decision_x = np.tile(decision,(no_state,1)).T
                #print(tiled_decision.shape)
                Eta_mutate = Eta_mutate_tmp * tiled_decision + (1 - tiled_decision) * Eta_mutate
                X_next = X_next_mt_sample * tiled_decision_x + (1 - tiled_decision_x) * X_next
                #print(decision.sum()/no_simu)
            c_var = c_var * c_var_updater_mh(decision.sum()/no_simu)
            #X_next = X_ft + Eta_mutate
            iterate = iterate +1
            lnp = np.log(np.mean(weights_lik))  + lnp
            #print(np.log(np.mean(weights_lik)))
            #lnp = np.log(np.sum(weights_val))  + lnp
            #print(lnp)
        X = X_next.copy()
    return lnp 

In [274]:
tempered_particle_filter(F,G,H,J,x_obs,Const_obs,5500,r_star = 2,phi_init = 0.00001,markov_time = 1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43


-262.65775503342263

In [226]:
tempered_particle_filter(F,G,H,J,x_obs,Const_obs,5500,r_star = 2,phi_init = 0.0002,markov_time = 1)

-265.6514201112559

In [248]:
linalg.eig(linalg.inv(J @ J.T))

(array([ 74.31629013+0.j,  11.55352912+0.j,   4.99137092+0.j]),
 array([[ 1.,  0.,  0.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  1.]]))

In [120]:
no_runs = 20
likelihood_val1 = np.zeros((no_runs,2))
for i in tqdm(range(no_runs)):
    t0 = time.time()
    likelihood_val1[i,0] = tempered_particle_filter(F,G,H,J,x_obs,Const_obs,7000,r_star = 2,phi_init = 0.002,markov_time = 1)
    t1 = time.time()
    total = t1-t0
    likelihood_val1[i,1] = t1-t0

  5%|██                                        | 1/20 [02:24<45:43, 144.40s/it]

KeyboardInterrupt: 

In [20]:
Bias1 = np.mean(likelihood_val1[0:15,0])+306.5
StDev1 = np.std(likelihood_val1[0:15,0])
AvgTime1 = np.mean(likelihood_val1[0:15,1])
print(Bias1, StDev1,AvgTime1)

-13.829112415 0.952899583544 278.806489515


In [21]:
likelihood_val1

array([[-321.79358071,  260.21936464],
       [-321.52966305,  257.07757258],
       [-319.74966799,  274.76480484],
       [-321.03674369,  270.34807229],
       [-318.77617059,  262.83290386],
       [-320.87006459,  244.65884948],
       [-320.95592647,  275.9442749 ],
       [-319.94505529,  288.21007514],
       [-319.71781955,  290.96917367],
       [-320.8293161 ,  294.37778306],
       [-320.26597819,  292.70448208],
       [-320.87039466,  294.1811583 ],
       [-318.10857379,  294.24260688],
       [-320.03536756,  291.18447828],
       [-320.45236399,  290.38174272],
       [-321.21540119,  292.31930947],
       [-321.97603628,  289.67436266],
       [-317.08295173,  290.12486768],
       [-319.08324511,  292.5842042 ],
       [-319.96450079,  287.21079969]])

Section on class model

In [195]:
param_true = np.array([1,0.2,0.99,1.25,0.25,0.5,0.75,0.75,1,1,2])
sigma_m = 0.1
no_st = 5
no_observables = 3
def eval_theta(param,sigma_m):
    sigma, kappa, beta , phi_pi,phi_x,rho_i,rho_g,rho_u,sigma_g,sigma_u,sigma_i = param
    no_st = 5 #Number of x states - underlying states
    #The X_t matrix
    A = np.reshape([1,0,sigma,-1,0,
                    -kappa,1,0,0,-1,
                    -phi_x*(1 - rho_i), - phi_pi * (1-rho_i),1,0,0,
                   0,0,0,1,0,
                    0,0,0,0,1],(no_st,no_st))
    #The X_t+1 matrix
    B = np.reshape([1, sigma, 0, 0, 0,
                  0, beta, 0, 0, 0,
                  0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0],(no_st,no_st))
    #The X_t-1 matrix
    C = np.reshape([0, 0, 0, 0, 0,
                    0, 0, 0, 0, 0,
                    0, 0, rho_i, 0, 0,
                    0, 0, 0, rho_g, 0,
                    0, 0, 0, 0, rho_u],(no_st,no_st))
    #The error matrix eta
    D = np.reshape([0, 0, 0,
                    0, 0, 0,
                    sigma_i, 0, 0,
                    0, sigma_g, 0,
                    0, 0, sigma_u],(no_st,3))
    K = np.bmat([[np.zeros((no_st,no_st)), np.eye(no_st)], [-C, A]])
    L = np.bmat([[ np.eye(no_st),np.zeros((no_st,no_st))], [np.zeros((no_st,no_st)), B]])
    T,S,alppha,betta,Q,Z = linalg.ordqz(K,L,sort='iuc')
    Z = Z.T
    #print(1)
    #F = Z[(no_st):(2*no_st),0:no_st] @ linalg.inv(Z[0:no_st,0:no_st])
    if (linalg.det(Z[0:no_st,0:no_st]) != 0):
        F = (linalg.solve(Z[0:no_st,0:no_st],Z[0:no_st,(no_st):(2*no_st)])).T
        #print(2)
        #G = np.linalg.inv(A - (B @ F)) @ D
        if(linalg.det(A - (B @ F)) != 0):
            G = linalg.solve(A - (B @ F),D)
        else:
            F = np.eye(no_st)
            G = F
    else:
        F = np.eye(no_st)
        G = F   
    H = np.eye(3,5) 
    J = sigma_m*np.eye(3,3)
    return F , G , H, J
def KF_filter(F,G,H,J,x_obs):
    X = np.zeros((5,1))  # Number of state variables
    P = sigma_m * np.eye(5,5)  #Initial variance guess
    lnp = 0
    G_quad = G @ G.T
    J_quad = J @ J.T
    I_d = np.eye(3,3)
    for t in range(x_obs.shape[0]):
        X = F @ X
        P = F @ P @ F.T + G_quad
        y = H @ X
        H_P_multi = H @ P
        V = H_P_multi @ H.T + J_quad
        inv_V = linalg.cho_solve(linalg.cho_factor(V,lower=True),I_d )
        x_obs_t_T = x_obs[t,None].T
        lnp = lnp - 0.5 * (np.log(np.linalg.det(V)) + (x_obs_t_T - y).T @ inv_V @ 
                           (x_obs_t_T-y))
        X = X + P @ H.T @ inv_V @ (x_obs_t_T - y)
        P = P - H_P_multi.T @ inv_V @ H_P_multi
    return lnp- 0.5 * x_obs.shape[0] * x_obs.shape[1] *  np.log(2*np.pi)  #With 2pi adjustment constant

In [196]:
T = 200
x = np.zeros((5,T)) 
v = np.random.randn(3,T)
F , G, H, J = eval_theta(param_true, sigma_m)
for t in range(T-1):
    x[:,t + 1] = F @ x[:,t] + G @ v[:,t + 1]
x_obs = np.zeros((T,3))
for t in range(T):
    x_obs[t,:] = H @ x[:,t] + J @ v[:,t]

In [197]:
KF_filter(F,G,H,J,x_obs)

array([[-1287.57807344]])

In [72]:
Var_y1 = J @ J.T
inv_Var_y1 = linalg.inv(Var_y1)
print(inv_Var_y1*0.002)

[[ 0.2  0.  -0. ]
 [ 0.   0.2 -0. ]
 [ 0.   0.   0.2]]


In [199]:
t0 = time.time()
Result1=particle_filter(F,G,H,J,x_obs,np.zeros((no_observables,1)),no_simu = 5000)
t1 = time.time()
total = t1-t0
print(Result1, total)

-2223.73084338 23.71008062362671


In [200]:
t0 = time.time()
Result1=tempered_particle_filter(F,G,H,J,x_obs,np.zeros((no_observables,1)),no_simu = 4000,r_star = 2,phi_init = 0.02,markov_time = 1)
t1 = time.time()
total = t1-t0
print(Result1, total)

-1589.68208288 242.73200821876526


In [107]:
t0 = time.time()
Result1=tempered_particle_filter(F,G,H,J,x_obs,np.zeros((no_observables,1)),no_simu = 11000,r_star = 2,phi_init = 0.02,markov_time = 1)
t1 = time.time()
total = t1-t0
print(Result1, total)

-1517.44462027 1771.5278310775757


In [194]:
def tempered_particle_filter(F,G,H,J,x_obs,Const_obs,no_simu,r_star = 2,phi_init = 0.2,markov_time = 1):  #F,G,H,J matrix from Kalman filter computation
    T = x_obs.shape[0]
    no_state = F.shape[0]
    no_measurement = J.shape[0]
    X_mean = np.zeros((no_st,1)) #Initialize the particle cloud at 0
    V_x = linalg.solve_discrete_lyapunov(F,G @ G.T) #Initial Variance function solved from FVF'−F+G @ G.T=0
    P = V_x# + 10 * np.eye(5,5) # P matrix is the initial variance
    lnp = 0 # Initialize likelihood value
    Eta = np.reshape(random_sample(np.zeros(no_state), G @ G.T,no_simu*T),(T,no_simu,no_state),order='F') 
    #Eta shock reshaped for the number similutaion and time periods, ordered by Fortran optimal code
    X = random_sample(X_mean.flatten(),P,no_simu) #Initial X draw
    p_yx = np.zeros((X.shape[0]))
    const_obs = np.tile(Const_obs.T,(no_simu,1))
    Var_y = J @ J.T
    Var_eps = G @ G.T 
    inv_Var_y = linalg.inv(Var_y)
    pinv_var_eps = linalg.pinv(Var_eps)
    maxiter = 20 # max number of iterations in the while loop
    accept_shock = np.random.uniform(size = (markov_time,no_simu,T,maxiter))
    mutate_shock = np.reshape(random_sample(np.zeros(no_state),np.eye(no_state),no_simu*T*maxiter*markov_time),(T,no_simu,no_state,maxiter,markov_time),order='F')
    def eval_weights(phi,phi_prev,e_dev):
        return (phi/phi_prev)**(no_measurement/2) * np.exp(-e_dev*(phi-phi_prev))
    def eval_inef_ratio(phi,phi_prev,e_dev,r_star):
        return np.mean(np.exp(-2*(phi-phi_prev) * e_dev))/(np.mean(np.exp(-(phi-phi_prev) * e_dev)))**2 - r_star
    def eval_weights_avg(eval_weights_tmp):
        return eval_weights_tmp/ np.mean(eval_weights_tmp)/no_simu
    def c_var_updater_mh(x):
        return 0.95 + 0.1 * np.exp(20 * (x-0.4))/(1 + np.exp(20 * (x-0.4)))
    
    for t in range(T): #Loop over time periods and add up the likelihood
        c_var = 0.3
        phi = phi_init
        #phi_tmp1 = phi_init
        #phi_tmp2 = phi_init
        X_ft = X @ F.T
        X_next = X_ft + Eta[t,:,:]             #Propagate the paricle forward
        y_dev = x_obs[t,:] -X_next @ H.T - const_obs
        e_dev = 0.5 * np.diag(( y_dev) @ inv_Var_y @ (y_dev).T)
        p_yx = np.exp(- phi* e_dev) * (np.linalg.det(inv_Var_y*phi))**(1/2)
        ##p_yx_true = np.exp( -0.5 * np.log(np.linalg.det(Var_y)) - e_dev) 
        #Computing the likelihoood
        p_yy = p_yx.mean()
        weights_val = p_yx/p_yy/no_simu #Giving weights to the observations
        multisample = np.random.multinomial(no_simu, weights_val.flatten(), size=1).flatten() #resampling based on weights
        X_next = np.repeat(X_next,multisample,axis = 0) #Creating a new sample based on the weights
        #X = X_next_sample.copy()
        iterate = 0
        lnp = np.log(p_yx.mean()) + lnp
        while (phi <1):
        #while (iterate < 0):
            #print(phi,phi_tmp1,phi_tmp2)
            y_dev = x_obs[t,:] -X_next @ H.T - const_obs
            e_dev = 0.5 * np.diag(( y_dev) @ inv_Var_y @ (y_dev).T)
            if(eval_inef_ratio(1,phi,e_dev,r_star) <= 0):
                weights_lik = eval_weights(1,phi,e_dev)
                #phi_prev = phi
                phi = 1
            else:
                #phi_tmp = optimize.fsolve(eval_inef_ratio,0.5,args = (phi,e_dev,r_star))
                phi_tmp = optimize.root(eval_inef_ratio,(phi +1)/2,args = (phi,e_dev,r_star))
                #phi_tmp = optimize.brentq(eval_inef_ratio,phi,1,args = (phi,e_dev,r_star))
                #phi_tmp = optimize.bisect(eval_inef_ratio,phi,1,args = (phi,e_dev,r_star))
                weights_lik = eval_weights(phi_tmp.x,phi,e_dev)
                phi = phi_tmp.x 
                #weights_lik = eval_weights(phi_tmp,phi,e_dev)
                #phi = phi_tmp
            weights_val = eval_weights_avg(weights_lik)
            #print(weights_val.min())
            #multisample = multinomial_robust(no_simu, weights_val.flatten())
            multisample = np.random.multinomial(no_simu, weights_val.flatten(), size=1).flatten() #resampling based on weights
            X_next = np.repeat(X_next,multisample,axis = 0) #Creating a new sample based on the weights
            Eta_mutate = X_next - X_ft
            for mt in range(markov_time):
                #print('runnnn')
                Eta_mutate_tmp = Eta_mutate + c_var * mutate_shock[t,:,:,iterate,mt]
                X_next_mt_sample = X_ft + Eta_mutate_tmp          #Propagate the paricle forward
                y_dev_mt_sample = x_obs[t,:] -X_next_mt_sample @ H.T - const_obs
                e_dev_mt_sample = 0.5 * np.diag(( y_dev_mt_sample) @ ( inv_Var_y) @ (y_dev_mt_sample).T)
                p_yx_tmp =  - phi* e_dev_mt_sample
                X_next_mt_past = X_ft + Eta_mutate             #Propagate the paricle forward
                y_dev_mt_past = x_obs[t,:] -X_next_mt_past @ H.T - const_obs
                e_dev_mt_past = 0.5 * np.diag(( y_dev_mt_past) @ ( inv_Var_y) @ (y_dev_mt_past).T)
                p_yx_past =  - phi* e_dev_mt_past
                pr_ratio = -0.5 * np.sum(Eta_mutate_tmp**2 -Eta_mutate**2,1)
                acceptance_rate = np.exp( p_yx_tmp - p_yx_past + pr_ratio)
                decision = acceptance_rate > accept_shock[mt,:,t,iterate]
                tiled_decision = np.tile(decision,(no_state,1)).T
                Eta_mutate = Eta_mutate_tmp * tiled_decision + (1 - tiled_decision) * Eta_mutate
                #print(decision.sum()/no_simu)
            c_var = c_var * c_var_updater_mh(decision.sum()/no_simu)
            X_next = X_ft + Eta_mutate
            iterate = iterate +1
            lnp = np.log(np.mean(weights_lik))  + lnp
            #lnp = np.log(np.sum(weights_val))  + lnp
            #print(lnp)
        X = X_next.copy()
        #print(t)
    return lnp- 0.5 * T * no_measurement *  np.log(2*np.pi)

In [195]:
tempered_particle_filter(F,G,H,J,x_obs,Const_obs,3000,r_star = 2,phi_init = 0.002,markov_time = 1)

-287.04619689891263